This notebook represents conceptual LegalRuleML encoding using rule-based formatting via the Pydantic Output Parser functionality of Llama. It can be used to generate categories of codes via descriptions in natural language. However, the limitation is that it may attach semantic meaning to those which are not strictly within a particular XML tag or concept within LegalRuleML

In [ ]:
!pip install langchain
!pip install pydantic
!pip install openai
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
!pwd

/content


In [ ]:
from secret_key import openai_key
openai_api_key=openai_key

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Union

class ConstitutiveRule(BaseModel):
    identifier: str = Field(..., description="Unique identifier for the rule")
    conditions: List[str] = Field(..., description="Conditions that create a new legal status or fact")
    result: str = Field(..., description="The legal status or fact created by the rule")

class PrescriptiveRule(BaseModel):
    identifier: str = Field(..., description="Unique identifier for the rule")
    preconditions: List[str] = Field(..., description="Conditions under which the rule applies")
    obligations: List[str] = Field(default=[], description="Obligations imposed by the rule")
    permissions: List[str] = Field(default=[], description="Permissions granted by the rule")
    prohibitions: List[str] = Field(default=[], description="Prohibitions enforced by the rule")

class LegalRule(BaseModel):
    type: Union[ConstitutiveRule, PrescriptiveRule] = Field(..., description="The specific type of legal rule, either constitutive or prescriptive")


In [ ]:
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser

# Assume initialization of OpenAI LLM and necessary imports

class LegalRuleParser(PydanticOutputParser):
    model = LegalRule

# Example usage
parser = LegalRuleParser()
prompt = '''Convert this into LegalRuleML format:- Responsibilities of dealers.
(1)
in the case the dealer has been given the responsibility of collection on behalf of the producer, the dealer shall collect the e-waste by providing the consumer a box, bin or a demarcated area to deposit e-waste, or through take back system and send the e-waste so collected to collection centre or dismantler or recycler as designated by producer;
(2)
the dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Scheme of the producer to the depositor of e-waste;
(3)
every dealer shall ensure that the e-waste thus generated is safely transported to authorised dismantlers or recyclers;
(4)
ensure that no damage is caused to the environment during storage and transportation of e-waste.'''
# Assuming `query` method exists and returns structured data compatible with our models
output = parser.parse({"type": {"identifier": "rule_1", "conditions": ["condition1"], "result": "legal fact"}})
output2 = parser.parse({"type": {"identifier": "rule_2", "preconditions": ["condition1"], "obligations": ["obligation1"], "permissions": [], "prohibitions": []}})


print(output)


ValidationError: 1 validation error for LegalRuleParser
pydantic_object
  field required (type=value_error.missing)

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import (
    PromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI
from typing import List, Optional, Union


# Define your desired data structure.

class ConstitutiveRule(BaseModel):
    identifier: str = Field(..., description="Unique identifier for the rule")
    conditions: List[str] = Field(..., description="Conditions that create a new legal status or fact")
    result: str = Field(..., description="The legal status or fact created by the rule")

class PrescriptiveRule(BaseModel):
    identifier: str = Field(..., description="Unique identifier for the rule")
    preconditions: List[str] = Field(..., description="Conditions under which the rule applies")
    obligations: List[str] = Field(default=[], description="Obligations imposed by the rule")
    permissions: List[str] = Field(default=[], description="Permissions granted by the rule")
    prohibitions: List[str] = Field(default=[], description="Prohibitions enforced by the rule")


class LegalRule(BaseModel):
    rule_type: Optional[str] = None  # This could be set based on user_choice or inferred from the data
    rule_data: Union[ConstitutiveRule, PrescriptiveRule, dict] = {}

    # Dynamically parse rule_data based on rule_type
    @validator('rule_data', pre=True, always=True)
    def parse_rule_data(cls, value, values):
        rule_type = values.get('rule_type')
        if rule_type == 'constitutive' and isinstance(value, dict):
            return ConstitutiveRule(**value)
        elif rule_type == 'prescriptive' and isinstance(value, dict):
            return PrescriptiveRule(**value)
        return value


# And a query intended to prompt a language model to populate the data structure.
conversion_query = '''Convert this text into LegalRuleML compliant code-  Responsibilities of dealers.
(1)
in the case the dealer has been given the responsibility of collection on behalf of the producer, the dealer shall collect the e-waste by providing the consumer a box, bin or a demarcated area to deposit e-waste, or through take back system and send the e-waste so collected to collection centre or dismantler or recycler as designated by producer;
(2)
the dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Scheme of the producer to the depositor of e-waste;
(3)
every dealer shall ensure that the e-waste thus generated is safely transported to authorised dismantlers or recyclers;
(4)
ensure that no damage is caused to the environment during storage and transportation of e-waste.'''

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=LegalRule)

# Prompt
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query", "user_choice"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Run
_input = prompt.format_prompt(query=conversion_query, user_choice="prescriptive")
model = OpenAI(openai_api_key=openai_api_key, temperature=0)
output = model(_input.to_string())
parser.parse(output)

LegalRule(rule_type='PrescriptiveRule', rule_data=PrescriptiveRule(identifier='Responsibilities of dealers', preconditions=['The dealer has been given the responsibility of collection on behalf of the producer'], obligations=['The dealer shall collect the e-waste by providing the consumer a box, bin or a demarcated area to deposit e-waste, or through take back system and send the e-waste so collected to collection centre or dismantler or recycler as designated by producer'], permissions=['The dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Scheme of the producer to the depositor of e-waste'], prohibitions=['The dealer shall ensure that the e-waste thus generated is safely transported to authorised dismantlers or recyclers', 'No damage shall be caused to the environment during storage and transportation of e-waste']))

In [ ]:
# And a query intended to prompt a language model to populate the data structure.
conversion_query = '''Convert this text into LegalRuleML compliant code-  7. Responsibilities of dealers.
(1)
in the case the dealer has been given the responsibility of collection on behalf of the producer, the dealer shall collect the e-waste by providing the consumer a box, bin or a demarcated area to deposit e-waste, or through take back system and send the e-waste so collected to collection centre or dismantler or recycler as designated by producer;
(2)
the dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Scheme of the producer to the depositor of e-waste;
(3)
every dealer shall ensure that the e-waste thus generated is safely transported to authorised dismantlers or recyclers;
(4)
ensure that no damage is caused to the environment during storage and transportation of e-waste.
8. Responsibilities of the refurbisher.
(1)
collect e-waste generated during the process of refurbishing and channelise the waste to authorised dismantler or recycler through its collection centre;
(2)
make an application in Form 1(a) in accordance with the procedure laid down in sub-rule (4) of rule 13 to the concerned State Pollution Control Board for grant of one time authorisation;
(a)
the concerned State Pollution Control Board shall authorise the Refurbisher on one time basis as per Form 1 (bb) and authorisation would be deemed as considered if not objected to within a period of thirty days;
(b)
the authorised Refurbisher shall be required to submit details of e-waste generated to the concerned State Pollution Control Board on yearly basis;
(3)
ensure that no damage is caused to the environment during storage and transportation of e-waste;
(4)
ensure that the refurbishing process do not have any adverse effect on the health and the environment;
(5)
ensure that the e-waste thus generated is safely transported to authorised collection centres or dismantlers or recyclers;
(6)
file annual returns in Form-3 to the concerned State Pollution Control Board, on or before the 30th day of June following the financial year to which that return relates;
(7)
maintain records of the e-waste handled in Form-2 and such records should be available for scrutiny by the appropriate authority.'''

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=LegalRule)

# Prompt
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query", "user_choice"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Run
_input = prompt.format_prompt(query=conversion_query, user_choice="prescriptive")
model = OpenAI(openai_api_key=openai_api_key, temperature=0)
output = model(_input.to_string())
parser.parse(output)

OutputParserException: Failed to parse LegalRule from completion 
{
    "rule_type": "Responsibilities of dealers",
    "rule_data": {
        "identifier": "7",
        "conditions": [
            "the dealer has been given the responsibility of collection on behalf of the producer"
        ],
        "result": "the dealer shall collect the e-waste by providing the consumer a box, bin or a demarcated area to deposit e-waste, or through take back system and send the e-waste so collected to collection centre or dismantler or recycler as designated by producer"
    }
},
{
    "rule_type": "Responsibilities of dealers",
    "rule_data": {
        "identifier": "7",
        "conditions": [
            "the dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Scheme of the producer to the depositor of e-waste"
        ],
        "result": "the dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Sche


```
# This is formatted as code
```



In [ ]:
print(parser.get_format_instructions())


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"rule_type": {"title": "Rule Type", "type": "string"}, "rule_data": {"title": "Rule Data", "default": {}, "anyOf": [{"$ref": "#/definitions/ConstitutiveRule"}, {"$ref": "#/definitions/PrescriptiveRule"}, {"type": "object"}]}}, "definitions": {"ConstitutiveRule": {"title": "ConstitutiveRule", "type": "object", "properties": {"identifier": {"title": "Identifier", "description": "Unique identifier for the rule", "type": "string"}, "conditions": {"title": "Conditions", "description": "Conditions that create a new legal status or fa

In [ ]:
print(_input)

text='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"rule_type": {"title": "Rule Type", "type": "string"}, "rule_data": {"title": "Rule Data", "default": {}, "anyOf": [{"$ref": "#/definitions/ConstitutiveRule"}, {"$ref": "#/definitions/PrescriptiveRule"}, {"type": "object"}]}}, "definitions": {"ConstitutiveRule": {"title": "ConstitutiveRule", "type": "object", "properties": {"identifier": {"title": "Identifier", "description": "Unique identifier for the rule", "type": "string"}, "conditions": {"title": "Conditions", "description": "Conditions

In [ ]:
# And a query intended to prompt a language model to populate the data structure.
conversion_query = '''Convert this text into LegalRuleML compliant code-  7. Responsibilities of dealers.
(1)
in the case the dealer has been given the responsibility of collection on behalf of the producer, the dealer shall collect the e-waste by providing the consumer a box, bin or a demarcated area to deposit e-waste, or through take back system and send the e-waste so collected to collection centre or dismantler or recycler as designated by producer;
(2)
the dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Scheme of the producer to the depositor of e-waste;
(3)
every dealer shall ensure that the e-waste thus generated is safely transported to authorised dismantlers or recyclers;
(4)
ensure that no damage is caused to the environment during storage and transportation of e-waste.
'''
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=LegalRule)

# Prompt
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query", "user_choice"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Run
_input = prompt.format_prompt(query=conversion_query, user_choice="prescriptive")
model = OpenAI(openai_api_key=openai_api_key, temperature=0)
output = model(_input.to_string())
parser.parse(output)

LegalRule(rule_type='PrescriptiveRule', rule_data=PrescriptiveRule(identifier='7', preconditions=['the dealer has been given the responsibility of collection on behalf of the producer'], obligations=['the dealer shall collect the e-waste by providing the consumer a box, bin or a demarcated area to deposit e-waste, or through take back system and send the e-waste so collected to collection centre or dismantler or recycler as designated by producer'], permissions=['the dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Scheme of the producer to the depositor of e-waste'], prohibitions=['ensure that no damage is caused to the environment during storage and transportation of e-waste']))

In [ ]:
print(output)


{
    "rule_type": "PrescriptiveRule",
    "rule_data": {
        "identifier": "7",
        "preconditions": [
            "the dealer has been given the responsibility of collection on behalf of the producer"
        ],
        "obligations": [
            "the dealer shall collect the e-waste by providing the consumer a box, bin or a demarcated area to deposit e-waste, or through take back system and send the e-waste so collected to collection centre or dismantler or recycler as designated by producer"
        ],
        "permissions": [
            "the dealer or retailer or e-retailer shall refund the amount as per take back system or Deposit Refund Scheme of the producer to the depositor of e-waste"
        ],
        "prohibitions": [
            "ensure that no damage is caused to the environment during storage and transportation of e-waste"
        ]
    }
}
